# Aula 03

---

* Tema da ementa
  * NOMES, VINCULAÇÕES E ESCOPO
* Tópico
  * 2.2 ESCOPO
* Objetivo
  * Entender o conceito de Escopo de uma linguagem e sua importância na manipulação de variáveis

Aula baseada no Capítulo 5 do livro: 

SEBESTA, Robert W. **Conceitos de Linguagens de Programação**. 11a Ed. Porto Alegre: Bookkman, 2018.

---

Um dos fatores mais importantes para o entendimento das variáveis é o `escopo`.

> O `escopo` de uma variável é a faixa de sentenças nas quais ela é visível.

> Uma variável é `visível` em uma sentença se ela pode ser **referenciada** ou **atribuída** nessa sentença.

O `escopo` mais comum é o **local**.

> Uma variável é **local** a uma unidade ou a um bloco de programa se for declarada lá. As variáveis não locais de uma unidade ou de um bloco de programa são aquelas visíveis dentro da unidade ou do bloco de programa, mas não declaradas nessa unidade ou nesse bloco.

## Escopo Estático

O `escopo estático` é a vinculação de nomes a variáveis não locais. Foi introduzido pelo ALGOL 60.

Existem duas categorias de linguagens de `escopo estático`: aquelas nas quais os subprogramas podem ser aninhados, as quais criam escopos estáticos aninhados, e aquelas nas quais os subprogramas não podem ser aninhados.

`Ada`, `JavaScript`, `Common Lisp`, `Scheme`, `Fortran 2003+`, `F#` e `Python` permitem subprogramas aninhandos, mas as linguagens baseadas em `C` não.

Exemplo (JavaScript):

```javascript
function big(){
  function sub1(){
    var x = 7;
    sub2();
  }
  function sub2(){
    var y = x;
  }
  var x = 3;
  sub1();
}
```

De onde vem a variável `x` no procedimento `sub2`?

A busca pelo `x` começa no procedimento no qual a referência ocorre, `sub2`, mas nenhuma declaração para `x` é encontrada lá. A busca continua no `pai estático` de `sub2`, que é `big`, onde a declaração de `x` é encontrada. O `x` declarado em `sub1` é ignorado porque não está nos `ancestrais estáticos` de `sub2`.

## Blocos

Os `blocos` são `escopos estáticos` definidos no meio do código executável. Esse conceito permite a uma seção de código ter suas próprias variáveis locais.

Os `escopos` criados por `blocos`, que podem ser aninhados em blocos maiores, são tratados exatamente como aqueles criados por subprogramas. Referências a variáveis em um `bloco` e que não estão declaradas lá são conectadas às declarações por meio da busca pelos `escopos` que o envolvem (`blocos` ou subprogramas), por ordem de tamanho (do menor para o maior).

Exemplo (C):

```c
void sub(){
  int count;
  ...
  while(...){
    int count;
    count++;
    ...
  }
}
```

No exemplo dado, a variável `count` dentro do `while` é local para o `while`, e não tem referência para a variável `count` declara no procedimento `sub`.

É necessário que o desenvolvedor conheça bem a linguagem que está utilizando. Por exemplo, o código do exemplo acima é válido em `C`e `C++`, porém é inválido em `Java` e `C#`, pois seus projetistas consideraram que o reúso de nomes em blocos aninhados era muito propenso a erros para ser permitido.

Enquanto isso, em `JavaScript`, blocos que não são função não podem ser definidos, mesmo que a linguagem utilize `escopo estático`.

## Escopo Global

Algumas linguagens (dentre elas o `Python`) permitem uma estrutura de programa que é uma sequência de definição de funções, nas quais as definições de variáveis podem aparecer fora das funções. Definições fora de funções em um arquivo criam `variáveis globais`, potencialmente visíveis para essas funções.

As regras de visibilidade para `variáveis globais` em `Python` não são usuais, pois as variáveis não são declaradas normalmente. Elas são implicitamente declaradas quando aparecem como alvos de sentenças de atribuição. Uma `variável global` pode ser referenciada em uma função, mas pode ter valores atribuídos a ela apenas se tiver sido declarada como global na função. Vejamos alguns exemplos:

In [5]:
dia = 'Segunda'

def teste():
    print('O dia global é:', dia)

teste()

O dia global é: Segunda


In [8]:
dia = 'Segunda'

def teste():
    print('O dia global é:', dia)
    dia = 'Terça'
    print('O novo valor do dia é:', dia)

teste()

UnboundLocalError: local variable 'dia' referenced before assignment

O erro aconteceu porque tentamos transformar a `variável global` `dia` em uma `variável local`.

Para informar ao interpretador do `Python` que queremos usar a mesma variável, podemos fazer da seguinte forma:

In [1]:
dia = 'Segunda'

def teste():
    global dia # indicando que é a variável global a ser manipulada
    print('O dia global é:', dia)
    dia = 'Terça'
    print('O novo valor do dia é:', dia)

teste()

O dia global é: Segunda
O novo valor do dia é: Terça


Quando temos funções aninhadas e utilizamos alguma variável não global, porém também não local àquele bloco, podemos utilizar a palavra reservada `nonlocal`.

In [6]:
def teste():
    x = 2

    def teste2():
        nonlocal x
        y = x + 1
        print(y)
    
    teste2()

teste()

3


## Escopo Dinâmico

O `escopo dinâmico` é baseado na **sequência de chamadas de subprogramas**, não em seu relacionamento espacial uns com os outros. Logo, o `escopo` pode ser determinado apenas em tempo de execução.

Revisitando o exemplo:

```javascript
function big(){
  function sub1(){
    var x = 7;
    sub2();
  }
  function sub2(){
    var y = x;
  }
  var x = 3;
  sub1();
}
```

Assumindo que as regras de `escopo dinâmico` se aplicam a referências não locais, o significado do identificador `x` referenciado em `sub2` é dinâmico. Quando a busca por declarações locais falha, as declarações do `pai dinâmico`, o procedimento que o chamou, são procuradas. Se não for encontrada, a busca continua pelos `ancestrais dinâmicos`.

No exemplo dado `big` chama `sub1`, o qual chama `sub2`. Como o `x` não está declarado em `sub2` a busca continua em seu chamador, `sub1`, onde existe a declaração. Dessa forma o `x` em `big` é ignorado.

## Escopo e Tempo de Vida

Considere o seguinte exemplo em C++:

```c++
void printheader(){
    ...
} /* fim de printheader */

void compute(){
    int sum;
    ...
    printheader();
} /* fim de compute */
```

O `escopo` da vairável `sum` é completamente contido na função `compute`. Ele não se estende até o corpo da função `printheader`, apesar de `printheader` executar no meio da execução de `compute`. Entretanto, o tempo de vida de `sum` se estende por todo o período em que `printheader` é executada.

---

## Exercícios

* [Lista 1](https://wiki.python.org.br/EstruturaDeDecisao)
* [Lista 2](https://wiki.python.org.br/EstruturaDeRepeticao)